# 🍎 Health Calculator Agent Tutorial 🍏

Welcome to the **Health Calculator Agent** tutorial, where we'll showcase how to:
1. **Initialize** a project and use the Azure AI Foundry ecosystem
2. **Create an Agent** with **Code Interpreter** capabilities
3. **Perform BMI calculations** and **analyze nutritional data** with sample CSV files
4. **Generate** basic health insights and disclaimers

> #### Ensure you have completed the [`1-basics.ipynb`](./1-basics.ipynb) notebook before starting this one.

## Let's Dive In
We'll walk step-by-step, similar to our **Fun & Fit** sample, but with a focus on using **Code Interpreter** for numeric calculations and data analysis. Let's begin!

<img src="./seq-diagrams/2-code-interpreter.png" width="30%"/>




## 🔐 Authentication Setup

Before running the next cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login --use-device-code
```

This will provide you with a device code and URL to authenticate in your browser, which is useful for:
- Remote development environments
- Systems without a default browser
- Corporate environments with strict security policies

After successful authentication, you can proceed with the notebook cells below.

## 1. Initial Setup
We'll start by importing libraries, loading environment variables, and initializing an **AIProjectClient**. We'll also create a sample CSV for demonstration.


In [ ]:
# Import required libraries
import os
import time
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv
from azure.identity import InteractiveBrowserCredential
from azure.ai.projects import AIProjectClient

# Load environment variables from the parent directory's .env
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir.parent / '.env')

# Get tenant ID and connection string
tenant_id = os.environ.get("TENANT_ID")
ai_foundry_project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")

print(f"🔑 Using Tenant ID: {tenant_id}")

# Initialize AIProjectClient with simplified browser-based authentication
try:
    print("🌐 Using browser-based authentication to bypass Azure CLI cache issues...")
    
    # Use only InteractiveBrowserCredential with the specific tenant
    credential = InteractiveBrowserCredential(tenant_id=tenant_id)
    
    # Create the project client using endpoint (ai_foundry_project_endpoint is actually the endpoint URL)
    project_client = AIProjectClient(
        endpoint=ai_foundry_project_endpoint,
        credential=credential
    )
    print("✅ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"❌ Error initializing client: {str(e)}")
    print("💡 Please complete the browser authentication prompt that should appear")

# Create sample CSV data for demonstration
def create_sample_data():
    try:
        data = {
            'Date': pd.date_range(start='2024-01-01', periods=7),
            'Calories': [2100, 1950, 2300, 2050, 1900, 2200, 2150],
            'Protein_g': [80, 75, 85, 78, 72, 82, 79],
            'Carbs_g': [250, 230, 270, 245, 225, 260, 255],
            'Fat_g': [70, 65, 75, 68, 63, 73, 71],
            'Fiber_g': [25, 22, 28, 24, 21, 26, 23]
        }
        df = pd.DataFrame(data)
        filename = os.environ.get("NUTRITION_DATA_FILENAME", "nutrition_data.csv")
        df.to_csv(filename, index=False)
        print(f"📄 Created sample data file: {filename}")
        return filename
    except Exception as e:
        print(f"❌ Error creating sample data: {e}")
        return None

sample_file = create_sample_data()

# Check what methods are available on the agents file client
print("\n🔍 Available file methods:")
if hasattr(project_client.agents, 'files'):
    file_methods = [method for method in dir(project_client.agents.files) if not method.startswith('_')]
    print(f"Files methods: {file_methods}")
else:
    print("Files attribute not found, checking agents methods...")
    agent_methods = [method for method in dir(project_client.agents) if not method.startswith('_')]
    print(f"Agent methods: {agent_methods}")

## 2. Create Health Calculator Agent 👩‍💻
We'll upload our sample CSV and then create an agent with **Code Interpreter** enabled. This agent can read the file, run Python code, and return results and visualizations.


In [ ]:
def create_health_calculator(file_path):
    """Create an agent with code interpreter for health/nutrition calculations."""
    try:
        print(f"📤 Uploading file: {file_path}")
        
        # Upload file with the working method
        uploaded_file = project_client.agents.files.upload_and_poll(
            file_path=file_path, 
            purpose="assistants"
        )
        print(f"✅ Uploaded CSV file, ID: {uploaded_file.id}")

        # Create the agent with code interpreter tool (without tool_resources to avoid SDK issues)
        agent = project_client.agents.create_agent(
            model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o"),
            name="health-calculator-agent",
            instructions="""
            You are a health calculator agent that can:
            1. Calculate and interpret BMI using the formula: BMI = weight(kg) / height(m)²
            2. Analyze provided nutrition data from CSV files that are attached to messages
            3. Generate charts and plots for data visualization
            4. Always include disclaimers that you are not a medical professional and cannot provide medical advice
            5. Use code interpreter to perform calculations and create visualizations
            
            When working with uploaded files:
            - Files will be attached to individual messages for you to analyze
            - You can access them directly using pandas or other Python libraries in code interpreter
            - Look for CSV files containing nutrition data with columns: Date, Calories, Protein_g, Carbs_g, Fat_g, Fiber_g
            
            When providing health information, always remind users that:
            - This is for educational purposes only
            - They should consult healthcare professionals for medical advice
            - Individual health needs vary significantly
            """,
            tools=[{"type": "code_interpreter"}]
            # Note: We'll attach files at the message level instead of agent level
        )
        print(f"🎉 Created health calculator agent, ID: {agent.id}")
        print(f"📄 File {uploaded_file.id} will be attached to individual messages when needed")
        return agent, uploaded_file
    except Exception as e:
        print(f"❌ Error creating health calculator agent: {e}")
        import traceback
        traceback.print_exc()
        return None, None

# Recreate the agent with the working approach
print("🔄 Recreating agent with message-level file attachment approach...")
health_agent, uploaded_file = None, None
if sample_file:
    health_agent, uploaded_file = create_health_calculator(sample_file)

## 3. BMI Calculation with Code Interpreter
We'll create a thread for BMI calculations. We'll feed in the user's height/weight, and ask the agent to show how it calculates BMI, interpret the result, and always disclaim professional advice.


In [ ]:
def calculate_bmi_with_agent(agent, height_inches, weight_pounds):
    """Calculate BMI using the code interpreter agent."""
    try:
        # Create a new conversation thread
        thread = project_client.agents.threads.create()
        print(f"📝 Created thread for BMI calculation, ID: {thread.id}")

        # Construct user message requesting BMI calculation
        user_text = (
            f"Calculate BMI for:\n"
            f"Height: {height_inches} inches\n"
            f"Weight: {weight_pounds} pounds\n\n"
            "Please:\n"
            "1. Show the calculation step by step\n"
            "2. Convert units to metric if needed\n"
            "3. Interpret the BMI result (underweight, normal, overweight, obese)\n"
            "4. Include appropriate health disclaimers\n"
            "5. Use Python code to perform the calculations"
        )

        # Create message in the thread
        msg = project_client.agents.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_text
        )
        print(f"➕ Created BMI request message, ID: {msg.id}")

        # Create and process the run using the simpler method we know works
        run = project_client.agents.create_thread_and_process_run(
            agent_id=agent.id,
            thread={
                "messages": [
                    {
                        "role": "user",
                        "content": user_text
                    }
                ]
            }
        )
        print(f"🤖 BMI run finished with status: {run.status}")
        return run.thread_id, run
    except Exception as e:
        print(f"❌ Error during BMI calculation: {e}")
        import traceback
        traceback.print_exc()
        return None, None

if health_agent:
    bmi_thread_id, bmi_run = calculate_bmi_with_agent(health_agent, 70, 180)  # example: 5'10" and 180 lbs

## 4. Nutrition Analysis
We'll create another thread where the user can ask the agent to analyze the nutrition data CSV we uploaded. The agent can read the file, compute macros, produce charts, and disclaim that it's not offering personalized medical advice.


In [ ]:
def analyze_nutrition_data(agent, uploaded_file):
    """Ask the agent to analyze the uploaded nutrition data by attaching it to the thread."""
    try:
        # Create a thread first
        thread = project_client.agents.threads.create()
        print(f"📝 Created thread for nutrition analysis, ID: {thread.id}")

        # Create user message with file attachment directly to the message
        user_text = (
            "Please analyze the attached nutrition CSV file.\n\n"
            "Please:\n"
            "1. Load and examine the data structure\n" 
            "2. Compute average daily macros (calories, protein, carbs, fat, fiber)\n"
            "3. Create visualizations to show trends over the week\n"
            "4. Identify any interesting patterns or insights\n"
            "5. Provide general health commentary with appropriate disclaimers\n"
            "6. Use Python code for all calculations and visualizations\n\n"
            "The CSV file contains daily nutrition data with columns: Date, Calories, Protein_g, Carbs_g, Fat_g, Fiber_g"
        )

        # Create message with file attachment
        message = project_client.agents.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_text,
            attachments=[{
                "file_id": uploaded_file.id,
                "tools": [{"type": "code_interpreter"}]
            }]
        )
        print(f"➕ Created message with file attachment, ID: {message.id}")

        # Now run the agent on this thread
        run = project_client.agents.runs.create_and_process(
            thread_id=thread.id,
            agent_id=agent.id
        )
        
        print(f"🤖 Nutrition run finished with status: {run.status}")
        return thread.id, run
    except Exception as e:
        print(f"❌ Error analyzing nutrition data: {e}")
        import traceback
        traceback.print_exc()
        return None, None

# Let's test the fixed nutrition analysis with file attachment
if health_agent and uploaded_file:
    print("\n🔄 Testing nutrition analysis with proper file attachment...")
    nutrition_thread_id_new, nutrition_run_new = analyze_nutrition_data(health_agent, uploaded_file)

## 5. Viewing Results & Visualizations 📊
The agent may produce text insights, disclaimers, and even images with charts. Let's fetch them from our threads!


In [ ]:
def view_agent_responses(thread_id):
    """View responses from the agent including text and any generated files."""
    try:
        print(f"\n🔎 Agent Responses from Thread: {thread_id}")
        print("=" * 60)
        
        # Get all messages from the thread
        messages = project_client.agents.messages.list(thread_id=thread_id)
        
        message_count = 0
        for msg in messages:
            message_count += 1
            role_emoji = "👤" if msg.role == "user" else "🤖"
            role_name = "User" if msg.role == "user" else "Agent"
            
            print(f"\n{message_count}. {role_emoji} {role_name}:")
            print("-" * 40)
            
            # Process message content
            if msg.content and len(msg.content) > 0:
                for content_item in msg.content:
                    if hasattr(content_item, "text") and content_item.text:
                        try:
                            content_text = content_item.text.value
                            # Clean the text to handle encoding issues
                            content_text = content_text.encode('utf-8', errors='ignore').decode('utf-8')
                            
                            # Truncate very long responses for readability
                            if len(content_text) > 1000:
                                print(content_text[:1000] + "\n... (truncated - see full output in notebook)")
                            else:
                                print(content_text)
                        except Exception as text_error:
                            print(f"❌ Error displaying text content: {text_error}")
                    
                    # Handle image files if any were generated
                    elif hasattr(content_item, "image_file"):
                        file_id = content_item.image_file.file_id
                        print(f"🖼️ Image generated with file ID: {file_id}")
                        try:
                            # Try to save the image
                            project_client.agents.files.save(
                                file_id=file_id, 
                                file_name=f"chart_{file_id}.png"
                            )
                            print(f"💾 Saved image as: chart_{file_id}.png")
                        except Exception as save_error:
                            print(f"❌ Could not save image: {save_error}")
        
        print(f"\n📊 Total messages in conversation: {message_count}")
        
    except Exception as e:
        print(f"❌ Error viewing agent responses: {e}")
        import traceback
        traceback.print_exc()

# Display BMI calculations
if 'bmi_thread_id' in locals() and bmi_thread_id:
    print("\n" + "="*80)
    print("🧮 BMI CALCULATION RESULTS")
    print("="*80)
    view_agent_responses(bmi_thread_id)

# Display original nutrition analysis (with issues)
if 'nutrition_thread_id' in locals() and nutrition_thread_id:
    print("\n" + "="*80)
    print("📊 ORIGINAL NUTRITION ANALYSIS RESULTS (Had File Access Issues)")
    print("="*80)
    view_agent_responses(nutrition_thread_id)

# Display new nutrition analysis (fixed)  
if 'nutrition_thread_id_new' in locals() and nutrition_thread_id_new:
    print("\n" + "="*80)
    print("📊 FIXED NUTRITION ANALYSIS RESULTS (With File Attachment)")
    print("="*80)
    view_agent_responses(nutrition_thread_id_new)

## 6. Cleanup & Best Practices
We can remove our agent and sample data if desired. In production, you might keep them for repeated usage.

### Best Practices in a Nutshell
1. **Data Handling** – Validate input data, handle missing values, properly manage file attachments.
2. **Calculations** – Provide formula steps, disclaimers, limit scope to general wellness, remind user you're not a doctor.
3. **Visualizations** – Use clear labeling and disclaimers that charts are for educational demonstrations.
4. **Security** – Monitor usage, limit access to code interpreter if dealing with proprietary data.


In [ ]:
def cleanup_all():
    """Clean up all created resources."""
    try:
        # Delete the uploaded CSV file from the service
        if 'uploaded_file' in globals() and uploaded_file:
            try:
                project_client.agents.files.delete(uploaded_file.id)
                print("🗑️ Deleted uploaded file from agent service.")
            except Exception as file_error:
                print(f"⚠️ Could not delete uploaded file: {file_error}")

        # Delete the agent if we created one
        if 'health_agent' in globals() and health_agent:
            try:
                project_client.agents.delete_agent(health_agent.id)
                print("🗑️ Deleted health calculator agent.")
            except Exception as agent_error:
                print(f"⚠️ Could not delete agent: {agent_error}")

        # Delete local CSV file
        if 'sample_file' in globals() and sample_file and os.path.exists(sample_file):
            try:
                os.remove(sample_file)
                print("🗑️ Deleted local sample CSV file.")
            except Exception as local_error:
                print(f"⚠️ Could not delete local file: {local_error}")

        print("\n✅ Cleanup completed!")
        
    except Exception as e:
        print(f"❌ Error during cleanup: {e}")
        import traceback
        traceback.print_exc()

# Option to run cleanup - you can comment this out if you want to keep the resources
print("🧹 Running cleanup to remove created resources...")
print("💡 Comment out the cleanup_all() call if you want to keep the agent and files for further testing.")
cleanup_all()

# Congratulations! 🎉

You've successfully completed the **Health Calculator Agent** with **Code Interpreter** tutorial! Here's what you accomplished:

## ✅ **What We Built**

### **🤖 Health Calculator Agent**
- Created an AI agent with **Code Interpreter** capabilities
- Enabled the agent to write and execute Python code in a secure sandbox
- Configured health-focused instructions with appropriate disclaimers

### **📊 Key Features Demonstrated**

1. **📁 File Upload & Management**
   - Uploaded CSV nutrition data to the Azure AI service
   - Learned correct file upload methods and purposes
   - **Fixed CSV file access issues** by using message-level file attachments

2. **🧮 BMI Calculations**
   - Agent performed step-by-step BMI calculations using Python code
   - Converted between imperial and metric units
   - Interpreted results with health categories
   - Provided responsible AI disclaimers

3. **📈 Nutrition Data Analysis**
   - Agent loaded and analyzed CSV data programmatically
   - Computed statistical averages for macronutrients
   - Generated data visualizations and trend analysis
   - Provided insights with appropriate health disclaimers

4. **🔧 Resource Management**
   - Properly cleaned up agents, files, and local resources
   - Demonstrated best practices for resource lifecycle management

## 🛠️ **Important Fix: CSV File Access Issue**

### **The Problem**
Initially, the agent couldn't access the uploaded CSV file, resulting in the error: *"It seems there was an issue locating the file."*

### **The Root Cause**
The issue was with how files were attached to the agent. Using `tool_resources` in the agent creation was causing SDK compatibility issues.

### **The Solution**
Instead of attaching files at the agent level, we attach them at the **message level**:

```python
# ❌ This caused issues:
# tool_resources={"code_interpreter": {"file_ids": [uploaded_file.id]}}

# ✅ This works:
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user", 
    content=user_text,
    attachments=[{
        "file_id": uploaded_file.id,
        "tools": [{"type": "code_interpreter"}]
    }]
)
```

### **Key Lessons**
- **Message-level attachments** are more reliable than agent-level tool_resources
- Always test file access in agents before proceeding with analysis
- The Azure AI SDK is evolving - message-level attachments are the current best practice

## 🎯 **Key Concepts Learned**

- **Code Interpreter Tool**: Enables agents to write and run Python code
- **File Integration**: How to upload and attach files to agent conversations
- **Secure Execution**: Code runs in a sandboxed environment
- **Data Analysis**: Agents can perform statistical analysis and create visualizations
- **Responsible AI**: Always include appropriate disclaimers for health-related content
- **Troubleshooting**: How to debug and fix file access issues

## 🔍 **API Methods Mastered**

- `project_client.agents.files.upload_and_poll()` - File upload with polling
- `project_client.agents.create_agent()` - Agent creation with tools
- `project_client.agents.messages.create()` - Creating messages with file attachments
- `project_client.agents.runs.create_and_process()` - Running agents with file access
- `project_client.agents.messages.list()` - Retrieving conversation history
- `project_client.agents.files.delete()` - Resource cleanup

## 🚀 **What's Next?**

Continue your Azure AI Agent Service journey with these advanced topics:

- **[3-file-search.ipynb](3-file-search.ipynb)** - Agents that can search through documents and knowledge bases
- **[4-bing_grounding.ipynb](4-bing_grounding.ipynb)** - Agents with real-time web search capabilities  
- **[5-agents-aisearch.ipynb](5-agents-aisearch.ipynb)** - Integration with Azure AI Search for enterprise knowledge
- **[6-agents-az-functions.ipynb](6-agents-az-functions.ipynb)** - Agents that can trigger Azure Functions and workflows

## 💡 **Best Practices Recap**

1. **File Attachment Strategy** - Use message-level file attachments instead of agent-level tool_resources for better compatibility
2. **Data Handling** - Validate inputs, handle missing values, manage file attachments properly
3. **Health Disclaimers** - Always remind users you're not a medical professional
4. **Resource Management** - Clean up agents and files to avoid unnecessary costs
5. **Error Handling** - Implement robust error handling for file operations and API calls
6. **Testing** - Always test file access before proceeding with analysis
7. **Security** - Use code interpreter for trusted operations only

## 🔍 **Troubleshooting Guide**

**If agents can't access uploaded files:**
1. ✅ Use message-level attachments instead of agent-level tool_resources  
2. ✅ Verify the file was uploaded successfully (check file ID)
3. ✅ Ensure the correct file purpose ("assistants") is used
4. ✅ Include file attachments in individual messages, not globally

Ready to explore more advanced agent capabilities? **Let's continue!** 🚀

---

*Happy (healthy) coding!* 💪